![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Context Types Deep Dive: Mastering the Building Blocks

## Welcome Back

You've now learned what context engineering is and understand the core concepts and constraints. You know about the 4 core components, the context window limitation, and the difference between static and dynamic context.

Now it's time to master each context type individually with detailed, hands-on examples and learn how to implement them effectively in your own systems.

## Learning Objectives

By the end of this notebook, you will:
1. Master each of the 4 context types with detailed examples
2. Implement context collection and management systems for each type
3. Measure the impact of each context type on AI performance
4. Design context strategies for different conversation patterns
5. Understand how context types interact and influence each other

## Setup

Let's start by importing the Redis Context Course models to work with clean, structured data:

In [1]:
import sys
import os
from datetime import datetime, time
from typing import List, Optional

# Add the reference agent to our path
sys.path.append('../../reference-agent')

try:
    from redis_context_course.models import (
        StudentProfile, Course, CourseRecommendation,
        DifficultyLevel, CourseFormat, Semester
    )
    print("✅ Successfully imported Redis Context Course models")
except ImportError as e:
    print(f"❌ Could not import models: {e}")
    print("Please ensure the reference-agent directory is available.")
    raise

✅ Successfully imported Redis Context Course models


## The Four Context Types

Let's explore each context type with practical examples using our Redis University course advisor.

## 1. System Context: The AI's Identity

System context defines what the AI knows about itself - its role, capabilities, and domain knowledge.

In [2]:
# Example: System context for our Redis University course advisor
system_context = """You are a Redis University course advisor. Your role is to help students 
choose the right Redis courses based on their background, goals, and preferences.

Available courses:
- RU101: Introduction to Redis (Beginner)
- RU201: Redis for Python (Intermediate, requires RU101)
- RU202: Redis for Java (Intermediate, requires RU101)
- RU301: Vector Similarity Search (Advanced, requires RU201 or RU202)
- RU302: Redis for Machine Learning (Advanced, requires RU301)

Always provide specific recommendations with clear reasoning."""

print("System Context Example:")
print(system_context)

System Context Example:
You are a Redis University course advisor. Your role is to help students 
choose the right Redis courses based on their background, goals, and preferences.

Available courses:
- RU101: Introduction to Redis (Beginner)
- RU201: Redis for Python (Intermediate, requires RU101)
- RU202: Redis for Java (Intermediate, requires RU101)
- RU301: Vector Similarity Search (Advanced, requires RU201 or RU202)
- RU302: Redis for Machine Learning (Advanced, requires RU301)

Always provide specific recommendations with clear reasoning.


**Key Characteristics of System Context:**
- **Static**: Doesn't change during conversations
- **Role-defining**: Establishes the AI's identity and capabilities
- **Domain-specific**: Contains knowledge about the subject area
- **Foundational**: Forms the base for all interactions

## 2. User Context: Personal Information

User context contains information about the specific user that enables personalization. Let's create a student profile using our structured models:

In [3]:
# Create a student profile using the StudentProfile model
sarah = StudentProfile(
    name="Sarah Chen",
    email="sarah.chen@university.edu",
    major="Computer Science",
    year=3,  # Junior
    completed_courses=["RU101"],
    current_courses=[],
    interests=["machine learning", "data science", "python"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE,
    max_credits_per_semester=15
)

print("Student Profile Example:")
print(f"Name: {sarah.name}")
print(f"Major: {sarah.major}, Year: {sarah.year}")
print(f"Completed: {sarah.completed_courses}")
print(f"Interests: {sarah.interests}")
print(f"Preferences: {sarah.preferred_format.value}, {sarah.preferred_difficulty.value} level")

Student Profile Example:
Name: Sarah Chen
Major: Computer Science, Year: 3
Completed: ['RU101']
Interests: ['machine learning', 'data science', 'python']
Preferences: online, intermediate level


**Key Characteristics of User Context:**
- **Personal**: Specific to individual users
- **Persistent**: Maintained across sessions
- **Evolving**: Updates as users progress and change
- **Enabling**: Makes personalization possible

## 3. Conversation Context: Memory and History

Conversation context maintains the flow of dialogue and enables the AI to understand references and follow-up questions.

In [4]:
# Example conversation history
conversation_history = [
    {"role": "user", "content": "What Redis course should I take next?"},
    {"role": "assistant", "content": "Based on your Python background and ML interests, I recommend RU201 (Redis for Python). You've completed RU101, so you meet the prerequisites."},
    {"role": "user", "content": "How long will that take to complete?"},
    {"role": "assistant", "content": "RU201 typically takes 6-8 hours to complete, with hands-on exercises included."},
    {"role": "user", "content": "What comes after that course?"}
]

print("Conversation Context Example:")
for i, message in enumerate(conversation_history, 1):
    role = message["role"].title()
    content = message["content"]
    print(f"{i}. {role}: {content}")

print("\nNote: The final question 'What comes after that course?' relies on conversation context.")
print("The AI knows 'that course' refers to RU201 from the previous exchange.")

Conversation Context Example:
1. User: What Redis course should I take next?
2. Assistant: Based on your Python background and ML interests, I recommend RU201 (Redis for Python). You've completed RU101, so you meet the prerequisites.
3. User: How long will that take to complete?
4. Assistant: RU201 typically takes 6-8 hours to complete, with hands-on exercises included.
5. User: What comes after that course?

Note: The final question 'What comes after that course?' relies on conversation context.
The AI knows 'that course' refers to RU201 from the previous exchange.


**Key Characteristics of Conversation Context:**
- **Temporal**: Ordered by time
- **Sequential**: Each message builds on previous ones
- **Growing**: Expands with each exchange
- **Reference-enabling**: Allows pronouns and implicit references

## 4. Retrieved Context: Dynamic Information

Retrieved context is information dynamically fetched from external sources based on the current query. Let's create some course data:

In [5]:
# Create course objects using the Course model
ru201 = Course(
    course_code="RU201",
    title="Redis for Python",
    description="Learn to use Redis with Python applications, including data structures, persistence, and performance optimization.",
    credits=3,
    difficulty_level=DifficultyLevel.INTERMEDIATE,
    format=CourseFormat.ONLINE,
    department="Computer Science",
    major="Computer Science",
    semester=Semester.FALL,
    year=2024,
    instructor="Dr. Python Expert",
    max_enrollment=50,
    current_enrollment=32,
    tags=["python", "redis", "databases", "performance"],
    learning_objectives=[
        "Connect Python applications to Redis",
        "Use Redis data structures effectively",
        "Implement caching strategies",
        "Optimize Redis performance"
    ]
)

print("Retrieved Context Example - Course Information:")
print(f"Course: {ru201.course_code} - {ru201.title}")
print(f"Level: {ru201.difficulty_level.value.title()}")
print(f"Format: {ru201.format.value.replace('_', ' ').title()}")
print(f"Enrollment: {ru201.current_enrollment}/{ru201.max_enrollment}")
print(f"Tags: {', '.join(ru201.tags)}")
print(f"Learning Objectives: {len(ru201.learning_objectives)} objectives defined")

Retrieved Context Example - Course Information:
Course: RU201 - Redis for Python
Level: Intermediate
Format: Online
Enrollment: 32/50
Tags: python, redis, databases, performance
Learning Objectives: 4 objectives defined


**Key Characteristics of Retrieved Context:**
- **Dynamic**: Fetched based on current needs
- **Query-specific**: Relevant to the current question
- **External**: Comes from databases, APIs, or knowledge bases
- **Fresh**: Can provide up-to-date information

## Context Integration: Bringing It All Together

In practice, all four context types work together to create intelligent responses. Let's see how they combine:

In [6]:
# Create a complete context example
def create_complete_context(student: StudentProfile, course: Course, conversation: list, system: str):
    """Combine all context types into a complete prompt"""
    
    # 1. System Context
    context_parts = [f"SYSTEM: {system}"]
    
    # 2. User Context
    user_info = f"""STUDENT PROFILE:
Name: {student.name}
Major: {student.major}, Year: {student.year}
Completed: {', '.join(student.completed_courses) if student.completed_courses else 'None'}
Interests: {', '.join(student.interests)}
Preferences: {student.preferred_format.value}, {student.preferred_difficulty.value} level"""
    context_parts.append(user_info)
    
    # 3. Retrieved Context
    course_info = f"""COURSE INFORMATION:
{course.course_code}: {course.title}
Level: {course.difficulty_level.value}
Format: {course.format.value}
Description: {course.description}
Learning Objectives: {'; '.join(course.learning_objectives)}"""
    context_parts.append(course_info)
    
    # 4. Conversation Context
    if conversation:
        conv_info = "CONVERSATION HISTORY:\n" + "\n".join(
            f"{msg['role'].title()}: {msg['content']}" for msg in conversation
        )
        context_parts.append(conv_info)
    
    return "\n\n".join(context_parts)

# Create complete context
complete_context = create_complete_context(
    student=sarah,
    course=ru201,
    conversation=conversation_history[:2],  # First 2 messages
    system=system_context
)

print("Complete Context Integration Example:")
print("=" * 50)
print(complete_context[:500] + "...")
print("=" * 50)
print("\nThis complete context would be sent to the LLM for generating responses.")

Complete Context Integration Example:
SYSTEM: You are a Redis University course advisor. Your role is to help students 
choose the right Redis courses based on their background, goals, and preferences.

Available courses:
- RU101: Introduction to Redis (Beginner)
- RU201: Redis for Python (Intermediate, requires RU101)
- RU202: Redis for Java (Intermediate, requires RU101)
- RU301: Vector Similarity Search (Advanced, requires RU201 or RU202)
- RU302: Redis for Machine Learning (Advanced, requires RU301)

Always provide specific reco...

This complete context would be sent to the LLM for generating responses.


## Context Management Strategies

Different scenarios require different context management approaches:

### Strategy 1: New User (Minimal Context)
- **System Context**: Full role definition
- **User Context**: Basic profile only
- **Conversation Context**: Empty
- **Retrieved Context**: General information

### Strategy 2: Returning User (Rich Context)
- **System Context**: Full role definition
- **User Context**: Complete profile with history
- **Conversation Context**: Recent conversation history
- **Retrieved Context**: Personalized, relevant information

### Strategy 3: Long Conversation (Optimized Context)
- **System Context**: Condensed role definition
- **User Context**: Key profile elements only
- **Conversation Context**: Summarized or recent messages only
- **Retrieved Context**: Highly relevant information only

## Key Takeaways

From this deep dive into context types, you now understand:

### The Four Context Types
1. **System Context**: Defines the AI's role and capabilities (static)
2. **User Context**: Personal information enabling personalization (persistent)
3. **Conversation Context**: Dialogue history maintaining flow (temporal)
4. **Retrieved Context**: Dynamic information from external sources (query-specific)

### Implementation Principles
- Use **structured data models** for clean, maintainable context
- **Combine all four types** for maximum effectiveness
- **Adapt strategies** based on user type and conversation length
- **Balance richness with efficiency** to manage token limits

### Next Steps
You're now ready to explore advanced context engineering techniques:
- **RAG (Retrieval-Augmented Generation)**: Advanced retrieved context
- **Memory Architecture**: Sophisticated conversation and user context
- **Context Optimization**: Efficient context management at scale

---

**Continue to Section 2: RAG Foundations**